# pandas中的分组操作
在数据分析中，分组操作是一种强大的数据摘要工具。通过分组，我们能够对数据集进行分类汇总和分析，这有助于我们理解数据的内在模式和关系。pandas提供的`groupby`功能是进行分组操作的核心。在本教程中，我们将深入探讨如何在pandas中使用分组，包括基本的分组操作、多重分组、以及如何使用聚合函数来提取分组后的统计信息。

### 分组操作的基本概念

在数据分析中，分组是一种非常重要的操作，它允许我们按照某些共同特征将数据划分为多个子集，并对每个子集执行特定的统计或计算操作。这种方法在各种数据分析任务中都非常常见，例如：

- 根据**地区**分组，然后计算每个地区的**销售总额**。
- 根据**品牌**分组，然后分析每个品牌的**市场份额**。
- 根据**客户等级**分组，确定不同等级客户的**平均消费额**。

这些场景说明了分组操作的一般模式，涉及到三个核心部分：**分组依据**（对哪个特征进行分组）、**数据来源**（要对哪些数据进行操作）、**操作及其返回结果**（要执行什么样的计算或统计任务）。

在 Pandas 中，分组操作通常遵循以下模式：

```
df.groupby(分组依据)[数据来源].操作()
```

如果我们有一个关于H&M客户的数据集，并想要按照用户ID统计每个用户的消费总金额，代码如下：

In [6]:
import numpy as np
import pandas as pd

df = pd.read_csv('./data/pandas_starter.csv')
df.groupby('customer_id')['price'].sum()

customer_id
03d0011487606c37c1b1ed147fc72f285a50c05f00b9712e0fc3da400c864296    49.967169
0bf4c6fd4e9d33f9bfb807bb78348cbf5c565846ff4006acf5c1b9aea77b0e54    30.655322
0d4fb6fb46dfe2759bcf7bc80340e8915b207aa2f74b5b3c76f74d3ce28359e8    24.071203
1320d4b3dd6481cde05bb80fb7ca37397f70470b9afb96aeca5d41175acaf836    38.350017
157eee38676eebb003bf97407f26e369de192997ab3902c194ce2690f060ff50    28.113153
15a075c3ea7b76dc31971a496c0d4b1f5f30170355eff7cedd4f4234c8c9402c    19.975017
1df07f916d7f648458702bd0b612caee88f1fb4cd1b660fc79ca0c99d27b1293    26.836746
1f09f1593c106b2b171e201a79e922f83ddacfdb690a0d8d382c2b7d03d0a5cb    19.664441
26fbbc7fb66c96e4d7984443dffa6779b9ac1e7f9c59c3207a24eacdd74e0891    24.319847
2df54f0d0653811fe06479c93905f3e6ecc6d07edf39d8b56e5b66c86182bedf    24.152373
30d1e9b6378a74a740f64c3d34f1686693d0430b03c6cd602d58062e604373d0    34.289932
3493c55a7fe252c84a9a03db338f5be7afbce1edbca12f3a908fac9b983692f2    16.290254
4308983955108b3af43ec57f0557211e44462a5633238351fff1

### 分组依据的本质
前面提到的若干例子都是以单一维度进行分组的，比如根据`customer_id`，如果现在需要根据多个维度进行分组，该如何做？事实上，只需在`groupby`中传入相应列名构成的列表即可。例如，现希望根据`customer_id`和`sales_channel_id`进行分组，统计`price`的均值就可以如下写出：

In [7]:
df.groupby(['customer_id', 'sales_channel_id'])['price'].sum()

customer_id                                                       sales_channel_id
03d0011487606c37c1b1ed147fc72f285a50c05f00b9712e0fc3da400c864296  1                    1.549780
                                                                  2                   48.417390
0bf4c6fd4e9d33f9bfb807bb78348cbf5c565846ff4006acf5c1b9aea77b0e54  1                    0.264237
                                                                  2                   30.391085
0d4fb6fb46dfe2759bcf7bc80340e8915b207aa2f74b5b3c76f74d3ce28359e8  1                    4.772763
                                                                                        ...    
e97c3a6c680cd3569df10f901a61fdffaf8f70300f6adf6e266b80c87d54245a  1                    5.707610
                                                                  2                   35.774441
efaafb08a00e63ce561a67c31c1ab7e720d4f394a78c4705518dc02084a29172  1                    4.512847
                                                     

目前为止，`groupby`的分组依据都是直接可以从列中按照名字获取的，那如果希望通过一定的复杂逻辑来分组，首先应该先写出分组条件：

In [8]:
condition = df['age']>30

In [9]:
condition

0        True
1        True
2        True
3        True
4        True
         ... 
50719    True
50720    True
50721    True
50722    True
50723    True
Name: age, Length: 50724, dtype: bool

In [10]:
df.groupby(condition)['price'].mean()

age
False    0.030844
True     0.031707
Name: price, dtype: float64

从索引可以看出，其实最后产生的结果就是按照条件列表中元素的值（此处是`True`和`False`）来分组，下面用随机传入字母序列来验证这一想法：

In [11]:
item = np.random.choice(list('abc'), df.shape[0])
item

array(['a', 'b', 'b', ..., 'c', 'b', 'b'], dtype='<U1')

In [12]:
item = np.random.choice(list('abc'), df.shape[0])
df.groupby(item)['price'].mean()

a    0.031617
b    0.031543
c    0.031211
Name: price, dtype: float64

此处的索引就是原先item中的元素，如果传入多个序列进入`groupby`，那么最后分组的依据就是这两个序列对应行的唯一组合：

In [13]:
df.groupby([condition, item])['price'].mean()

age     
False  a    0.030977
       b    0.031027
       c    0.030533
True   a    0.031872
       b    0.031750
       c    0.031493
Name: price, dtype: float64

### Groupby对象
当我们调用`df.groupby()`时，实际上创建了一个`GroupBy`对象。这个对象有许多方法和属性，可以帮助我们执行分组操作。

创建一个GroupBy对象：

In [14]:
gb = df.groupby(['customer_id', 'sales_channel_id'])
gb

通过`ngroups`属性，可以得到分组个数：

In [15]:
gb.ngroups

88

通过`groups`属性，可以返回从$\color{#FF0000}{组名}$映射到$\color{#FF0000}{组索引列表}$的字典：

In [16]:
res = gb.groups
res.keys() # 字典的值由于是索引，元素个数过多，此处只展示字典的键

dict_keys([('03d0011487606c37c1b1ed147fc72f285a50c05f00b9712e0fc3da400c864296', 1), ('03d0011487606c37c1b1ed147fc72f285a50c05f00b9712e0fc3da400c864296', 2), ('0bf4c6fd4e9d33f9bfb807bb78348cbf5c565846ff4006acf5c1b9aea77b0e54', 1), ('0bf4c6fd4e9d33f9bfb807bb78348cbf5c565846ff4006acf5c1b9aea77b0e54', 2), ('0d4fb6fb46dfe2759bcf7bc80340e8915b207aa2f74b5b3c76f74d3ce28359e8', 1), ('0d4fb6fb46dfe2759bcf7bc80340e8915b207aa2f74b5b3c76f74d3ce28359e8', 2), ('1320d4b3dd6481cde05bb80fb7ca37397f70470b9afb96aeca5d41175acaf836', 1), ('1320d4b3dd6481cde05bb80fb7ca37397f70470b9afb96aeca5d41175acaf836', 2), ('157eee38676eebb003bf97407f26e369de192997ab3902c194ce2690f060ff50', 1), ('157eee38676eebb003bf97407f26e369de192997ab3902c194ce2690f060ff50', 2), ('15a075c3ea7b76dc31971a496c0d4b1f5f30170355eff7cedd4f4234c8c9402c', 1), ('15a075c3ea7b76dc31971a496c0d4b1f5f30170355eff7cedd4f4234c8c9402c', 2), ('1df07f916d7f648458702bd0b612caee88f1fb4cd1b660fc79ca0c99d27b1293', 1), ('1df07f916d7f648458702bd0b612caee88f1fb

## 聚合函数

### 内置聚合函数

在`GroupBy`对象上已经定义了许多内置的聚合函数，这些函数通常经过优化，能够提供更快的性能。这些函数按照返回标量值的原则，包括：`max`, `min`, `mean`, `median`, `count`, `all`, `any`, `idxmax`, `idxmin`, `mad`, `nunique`, `skew`, `quantile`, `sum`, `std`, `var`, `sem`, `size`, `prod`等。

例如，我们可以获取每个年龄组的最大价格：

In [17]:
gb = df.groupby('age')['price']

In [18]:
gb.max()

age
22.0    0.118627
23.0    0.167797
24.0    0.101678
25.0    0.252542
26.0    0.101678
27.0    0.506780
28.0    0.218644
30.0    0.098288
31.0    0.303390
32.0    0.337288
33.0    0.303390
36.0    0.239898
40.0    0.174915
42.0    0.101678
43.0    0.167797
44.0    0.152525
45.0    0.167797
46.0    0.167797
47.0    0.167797
49.0    0.152525
51.0    0.337288
52.0    0.201695
53.0    0.422034
54.0    0.167797
55.0    0.118627
58.0    0.085169
59.0    0.213390
60.0    0.218644
61.0    0.218644
64.0    0.084729
67.0    0.084729
68.0    0.252542
70.0    0.252542
Name: price, dtype: float64

In [19]:
gb.quantile(0.95)

age
22.0    0.050831
23.0    0.067780
24.0    0.050831
25.0    0.059305
26.0    0.064605
27.0    0.118627
28.0    0.067780
30.0    0.050831
31.0    0.067780
32.0    0.067780
33.0    0.072017
36.0    0.067780
40.0    0.054220
42.0    0.067780
43.0    0.067780
44.0    0.064136
45.0    0.067780
46.0    0.064847
47.0    0.095102
49.0    0.059305
51.0    0.084729
52.0    0.084729
53.0    0.084729
54.0    0.081339
55.0    0.059305
58.0    0.038475
59.0    0.059305
60.0    0.067780
61.0    0.084729
64.0    0.050831
67.0    0.034080
68.0    0.067780
70.0    0.135576
Name: price, dtype: float64

### agg方法

虽然`GroupBy`对象提供了多个便捷的聚合方法，但有时我们需要更灵活的操作，比如：

* 同时使用多个聚合函数
* 对特定列使用特定的聚合函数
* 使用自定义的聚合函数
* 在聚合前自定义结果列名

`agg`方法可以帮助我们解决这些问题：

【a】同时使用多个函数

使用多个聚合函数时，将它们作为列表传递给`agg`方法：

In [21]:
gb = df.groupby('article_id')[['price','age']]
gb.agg(['min','max','mean'])

price                       age                 
                 min       max      mean   min   max       mean
article_id                                                     
108775015   0.006763  0.008458  0.007612  28.0  61.0  41.125000
108775044   0.006763  0.008458  0.008119  22.0  51.0  40.600000
110065001   0.025407  0.025407  0.025407  23.0  55.0  48.600000
110065011   0.010831  0.025407  0.017746  31.0  58.0  48.600000
111565001   0.008458  0.008458  0.008458  22.0  22.0  22.000000
...              ...       ...       ...   ...   ...        ...
946763001   0.033881  0.033881  0.033881  70.0  70.0  70.000000
946795001   0.042356  0.042356  0.042356  23.0  23.0  23.000000
947509001   0.030492  0.030492  0.030492  32.0  33.0  32.333333
952267001   0.016932  0.016932  0.016932  68.0  68.0  68.000000
956217002   0.059305  0.059305  0.059305  31.0  31.0  31.000000

[22334 rows x 6 columns]

【b】对特定的列使用特定的聚合函数

当我们需要对不同的列使用不同的聚合函数时，可以传递一个字典到`agg`方法：

In [29]:
gb.agg({'price':['min','max'], 'age':'median'})

price              age
                 min       max median
article_id                           
108775015   0.006763  0.008458   46.0
108775044   0.006763  0.008458   46.0
110065001   0.025407  0.025407   55.0
110065011   0.010831  0.025407   55.0
111565001   0.008458  0.008458   22.0
...              ...       ...    ...
946763001   0.033881  0.033881   70.0
946795001   0.042356  0.042356   23.0
947509001   0.030492  0.030492   32.0
952267001   0.016932  0.016932   68.0
956217002   0.059305  0.059305   31.0

[22334 rows x 3 columns]

【c】使用自定义的聚合函数

`agg` 方法还允许使用自定义函数进行聚合。这在内置的聚合函数无法满足特定需求时非常有用：

In [30]:
# 自定义聚合函数
def range_func(series):
    return series.max() - series.min()

# 应用自定义聚合函数
agg_custom = gb.agg({'price': range_func, 'age': 'mean'})
agg_custom

,price,age
article_id,,
108775015,0.001695,41.125000
108775044,0.001695,40.600000
110065001,0.000000,48.600000
110065011,0.014576,48.600000
111565001,0.000000,22.000000
...,...,...
946763001,0.000000,70.000000
946795001,0.000000,23.000000
947509001,0.000000,32.333333


在这个例子中，我们定义了一个 `range_func` 函数来计算价格的范围（最大值减最小值），并将其应用于 `price` 列，同时计算 `age` 列的平均值。

【d】在聚合前自定义结果列名

如果希望在聚合之前指定结果的列名，可以在聚合函数中使用元组。这样可以在结果 DataFrame 中直接得到更有意义的列名：

In [33]:
# 使用元组自定义结果列名
agg_named = gb.agg(price_range=('price', range_func),
                   age_average=('age', 'mean'),
                   age_max=('age','max'))
agg_named

,price_range,age_average,age_max
article_id,,,
108775015,0.001695,41.125000,61.0
108775044,0.001695,40.600000,51.0
110065001,0.000000,48.600000,55.0
110065011,0.014576,48.600000,58.0
111565001,0.000000,22.000000,22.0
...,...,...,...
946763001,0.000000,70.000000,70.0
946795001,0.000000,23.000000,23.0
947509001,0.000000,32.333333,33.0
